# Phenotype-based Machine Learning Approach for Disease Prediction

In [5]:
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
import os
import sys
import logging
mf_module_path = os.path.abspath(os.path.join('../python'))
if mf_module_path not in sys.path:
    sys.path.append(mf_module_path)
import mf
import mf_random
import hpoutil
import networkx
import obonet
import pickle
from tqdm import tqdm
import time

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [6]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

2019-09-26 09:46:14,781 - 3919 - root - DEBUG - test


In [7]:
mydb = mysql.connector.connect(host='localhost',
                               user='mimicuser',
                               passwd='mimic',
                               database='mimiciiiv13',
                              auth_plugin='mysql_native_password')
cursor = mydb.cursor(buffered=True)

## Task 1: Phenome-wide association studies for diseases

## Data selection
We adopt a similar approach to the loinc2hpo paper. At the patient level, find whether they have a diagnosis code or not, find whether they have a phenotype or not, and then perform logistic regression to determine whether a phenotype is associated with a disease. Control gender and age. 

We will create three separate tables:

    * diagnosis table (first diagnosis): subject_id, ICD9_code_value
    * phenotype table (phenotypes before diagnosis): subject_id, HP terms
    * patient table: subject_id, sex, age at diagnosis
    
    
We will create the tables gradually. First step, we use a simplied version: ignore date, just find diagnosis, and phenotypes for each patient, and test out the whole process

In [9]:
def patient_vectors():
    """
    Find information of patients that showed at least one encounter during defined period
    """
    return pd.read_sql_query('''
        SELECT 
            PATIENTS.SUBJECT_ID, PATIENTS.GENDER, PATIENTS.DOB
        FROM 
            PATIENTS
        WHERE 
            SUBJECT_ID IN (SELECT SUBJECT_ID FROM JAX_encounterOfInterest) 
    ''')

def diagnosis_vectors(icd):
    """
    same as createDiagnosisTable()
    """
    pd.read_sql_query('''
        SELECT
            SUBJECT_ID, HADM_ID, DIAGNOSIS
        FROM
            JAX_mf_diag
    ''', mydb)

def phenotype_vectors():
    pd.read_sql_query('''
        SELECT SUBJECT_ID, MAP_TO, dummy
        FROM JAX_LABHPOPROFILE
        GROUP BY SUBJECT_ID, MAP_TO
        HAVING COUNT(*) > 3
    ''')

In [18]:
initTables(debug=True)
patient_vectors()



0